In [2]:
import glob
import os
import pandas as pd
from Bio import SeqIO
from natsort import natsorted
import subprocess
import numpy as np
import re
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Alphabet import IUPAC
from Bio.Alphabet import generic_dna
# import arrow
from Bio import Phylo
from Bio import SeqIO
import matplotlib.pyplot as plt
import time
import string
from pyfasta import Fasta
%matplotlib inline

**Identifying 16S genes with Barrnap**

In [5]:
!mkdir -p /Users/tiagoferreiraleao/Dropbox/tiago-NAS/cyanobiome/barrnap/

def run_barrnap(inputs_folder,output_file,test=False):
    start_time = time.time()
    if os.path.exists(output_file):
        print("Barrnap output already exists")
    else:
        if test == True:
            input_list = glob.glob("%s*.fasta"%inputs_folder)[:5]
        else:
            input_list = glob.glob("%s*.fasta"%inputs_folder)
        count = 0
        for item in input_list:
            count += 1
            cmd = "barrnap --threads 12 %s | cat >> %s"%(item,output_file)
            subprocess.call(cmd,shell=True)
        print('\n' + "Barrnap run finished! %s genomes probed. Result saved as %s."%(count,output_file))
        print('\n' + "--- %s seconds ---" %(time.time()-start_time))


run_barrnap("/Volumes/TFL190831/cyanobiome_final_scaffolds/",
            "/Users/tiagoferreiraleao/Dropbox/tiago-NAS/cyanobiome/barrnap/cyanobiome_rRNA.csv")

Barrnap output already exists


**Extracting 16S genes**

In [6]:
def split_string_at(s, c, n):
    words = s.split(c)
    return c.join(words[:n]), c.join(words[n:])

def extract_nucleotides(fastafile,contig_name,start,end):
    if os.path.exists(fastafile):
        f = Fasta(fastafile)
        nucleotides = f[str(contig_name)][start:end]
    else:
        raise ValueError("Fasta file not found")
    return nucleotides

def inspect_fasta(fastafile):
    if os.path.exists(fastafile):
        input_handle = open(fastafile, "rU")
        for record in SeqIO.parse(input_handle, "fasta"):
            if len(record.id) == 0:
                raise ValueError("Fasta file contain blank header")
            if re.match('^[ATGCURYSWKMBVDHN]*$','%s'%record.seq):
                continue
            else:
                raise ValueError("Fasta file contain unknown nucleotide")
        input_handle.close()
    else:
        raise ValueError("Fasta file not found")
    return True

def rename_duplicated_fasta_headers(input_file):
    outname = "%s.ren.fasta"%os.path.splitext(input_file)[0]
    if os.path.exists(outname):
        print("Renamed fasta file already exists")
    else:
        header_list = []
        new_file = []
        if inspect_fasta(input_file) == True:
            with open(input_file, "rU") as handle:
                for record in SeqIO.parse(handle, "fasta"):
                    header_list.append(record.id)
                    count = header_list.count(record.id)
                    record.id = "%s_%s"%(record.id,string.ascii_uppercase[count-1])
                    record.description = ""
                    new_file.append(record)
                output_handle = open(outname, "w")
                SeqIO.write(new_file, output_handle, "fasta")
                output_handle.close()

def fetch_barrnap_rRNA_genes(input_dir,barrnap_df,rRNA,output_file):
    rRNA_options = ['5S','16S','23S']
    if rRNA not in rRNA_options:
        raise ValueError("Select '5S', '16S' or '23S' as the rRNA to be fetched")
    else:
        records = []
        for i,r in barrnap_df.iterrows():
            m = re.search(r'%s'%rRNA,barrnap_df.i.loc[i])
            if m:
                strain_name = 'SIO' + split_string_at(barrnap_df.a.loc[i],'_',1)[0]
                fastafile = "%s%s.fasta"%(input_dir,strain_name)
                if os.path.exists(fastafile):
                    nucleotides = extract_nucleotides(fastafile,barrnap_df.a.loc[i],
                                                      int(barrnap_df.d.loc[i]),int(barrnap_df.e.loc[i]))
                    seq = SeqRecord(Seq(nucleotides,generic_dna), id = "%s"%barrnap_df.a.loc[i], description="%s_gene"%rRNA)
                    records.append(seq)
                else:
                    raise ValueError("Fasta file %s not found"%fastafile)
    handle = open("%s"%output_file, "w")
    SeqIO.write(records,handle,"fasta")
    handle.close
    return records

def extract_rRNA_genes(df,barrnap_input,genomes_folder,output_file,rRNA):
    if os.path.exists(output_file):
        print("%s genes fasta output already exists"%rRNA)
    else:
        start_time = time.time()
        records = fetch_barrnap_rRNA_genes(genomes_folder,df,rRNA,output_file)
        rename_duplicated_fasta_headers(output_file)
        count = len(records)
        print("%s %s genes extracted! Fasta files saved as %s and %s.ren.fasta."%(count,rRNA,output_file,
                                                                                  os.path.splitext(output_file)[0]))
        print('\n' + "--- %s seconds ---" %(time.time()-start_time))

In [7]:
barrnap_file = "/Users/tiagoferreiraleao/Dropbox/tiago-NAS/cyanobiome/barrnap/cyanobiome_rRNA.csv"
barrnap_df = pd.read_csv(barrnap_file,names=list(string.ascii_lowercase)[0:9],sep='\t').dropna()

extract_rRNA_genes(barrnap_df,barrnap_file,
                   "/Volumes/TFL190831//cyanobiome_final_scaffolds/",
                   "/Users/tiagoferreiraleao/Dropbox/tiago-NAS/cyanobiome/barrnap/cyanobiome_16S.fasta","16S")

16S genes fasta output already exists


**Classifying 16S sequences with RDP**

```
- File "cyanobiome_16S.ren.fasta" was submitted to RDP, generating "allrank_cyanobiome_16S.ren.fasta_classified.txt"
```

In [8]:
def get_rdp_df(rdp_result):
    if os.path.exists(rdp_result):
        rdp_df = pd.read_csv(rdp_result,sep=';',skiprows=7,names=list(string.ascii_lowercase)[0:14])
        rdp_df = rdp_df.stack().str.replace(' ', '_').unstack()
    else:
        raise ValueError("RDP-classifier file not found")
    return rdp_df

rdp_df = get_rdp_df("/Users/tiagoferreiraleao/Dropbox/tiago-NAS/cyanobiome/barrnap/allrank_cyanobiome_16S.ren.fasta_classified.txt")

rdp_df[:5]

,a,b,c,d,e,f,g,h,i,j,k,l,m,n
0,3G5_NODE_8_A,-,Root,100%,Bacteria,100%,Cyanobacteria/Chloroplast,100%,Cyanobacteria,100%,Family_I,79%,Potamolinea,59%
1,1F9_NODE_11_A,+,Root,100%,Bacteria,100%,Cyanobacteria/Chloroplast,100%,Cyanobacteria,100%,Family_XII,100%,GpXII,100%
2,2C4_NODE_10_A,-,Root,100%,Bacteria,100%,Cyanobacteria/Chloroplast,100%,Cyanobacteria,100%,Family_I,72%,Potamolinea,36%
3,2C4_NODE_201_A,-,Root,100%,Archaea,71%,Crenarchaeota,44%,Thermoprotei,44%,Thermoproteales,20%,Thermoproteaceae,20%
4,1G2_NODE_259_A,-,Root,100%,Archaea,67%,Crenarchaeota,43%,Thermoprotei,43%,Thermoproteales,33%,Thermoproteaceae,33%


**Subsetting Cyanobacteria**

In [9]:
rdp_df_filt = rdp_df[rdp_df.g == "Cyanobacteria/Chloroplast"]
rdp_df_filt.reset_index(inplace=True)

rdp_df_filt[:5]

,index,a,b,c,d,e,f,g,h,i,j,k,l,m,n
0,0,3G5_NODE_8_A,-,Root,100%,Bacteria,100%,Cyanobacteria/Chloroplast,100%,Cyanobacteria,100%,Family_I,79%,Potamolinea,59%
1,1,1F9_NODE_11_A,+,Root,100%,Bacteria,100%,Cyanobacteria/Chloroplast,100%,Cyanobacteria,100%,Family_XII,100%,GpXII,100%
2,2,2C4_NODE_10_A,-,Root,100%,Bacteria,100%,Cyanobacteria/Chloroplast,100%,Cyanobacteria,100%,Family_I,72%,Potamolinea,36%
3,6,2G3_NODE_14_A,+,Root,100%,Bacteria,100%,Cyanobacteria/Chloroplast,100%,Cyanobacteria,100%,Family_I,100%,Potamolinea,100%
4,7,3C2_NODE_20_A,+,Root,100%,Bacteria,100%,Cyanobacteria/Chloroplast,100%,Cyanobacteria,100%,Family_IV,94%,GpIV,94%


**Saving 16S rRNA genes**

In [46]:
okeania_list = ['2E6','1B1','2D2','3E6','1A3','2C1','1C4','3C6']

In [47]:
new_file = []

input_handle = open("/Users/tiagoferreiraleao/Dropbox/tiago-NAS/cyanobiome/barrnap/cyanobiome_16S.ren.fasta", "rU")
outfile = "/Users/tiagoferreiraleao/Dropbox/tiago-NAS/cyanobiome/barrnap/cyanobiome_16S.filtered.fasta"

node_count = 0
for record in SeqIO.parse(input_handle, "fasta"):
    if record.id in list(rdp_df_filt["a"]) and record.id[:3] in okeania_list:
        print(record.id)
        node_count += 1
        new_file.append(record)
output_handle = open(outfile, "w")
SeqIO.write(new_file, output_handle, "fasta")
output_handle.close()
input_handle.close()
print("%s contigs renamed"%(node_count))

1C4_NODE_2_A
3E6_NODE_25_A
2 contigs renamed


/Users/tiagoferreiraleao/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 'U' mode is deprecated
  This is separate from the ipykernel package so we can avoid doing imports until


**Adding NCBI 16S rRNA sequences**

In [25]:
%%bash

cd /Users/tiagoferreiraleao/Dropbox/tiago-NAS/cyanobiome/barrnap/

cat NCBI_symploca_16S_tree/*.txt > ./NCBI_16S.fasta

cat NCBI_16S.fasta cyanobiome_16S.filtered.fasta > NCBI_cyanobiome_16S.fasta

**Running MUSCLE and trimAl**

In [26]:
#downloading trimAl
# !wget http://trimal.cgenomics.org/_media/trimal.v1.2rev59.tar.gz -P /Users/tiagoferreiraleao/Dropbox/tiago-NAS/cyanobiome/barrnap/
# !tar -xzf /Users/tiagoferreiraleao/Dropbox/tiago-NAS/cyanobiome/barrnap/trimal.v1.2rev59.tar.gz -C /Users/tiagoferreiraleao/Dropbox/tiago-NAS/cyanobiome/barrnap/; rm /Users/tiagoferreiraleao/Dropbox/tiago-NAS/cyanobiome/barrnap/trimal.v1.2rev59.tar.gz

In [27]:
%%bash

cd /Users/tiagoferreiraleao/Dropbox/tiago-NAS/cyanobiome/barrnap/trimAl/source/

make #compiling trimAl

g++ -Wall  -o readal readAl.cpp -lm alignment.o statisticsGaps.o utils.o similarityMatrix.o statisticsConservation.o sequencesMatrix.o compareFiles.o
g++ -Wall  -o trimal  main.cpp -lm alignment.o statisticsGaps.o utils.o similarityMatrix.o statisticsConservation.o sequencesMatrix.o compareFiles.o


In [28]:
!muscle -in /Users/tiagoferreiraleao/Dropbox/tiago-NAS/cyanobiome/barrnap/NCBI_cyanobiome_16S.fasta -out /Users/tiagoferreiraleao/Dropbox/tiago-NAS/cyanobiome/barrnap/NCBI_cyanobiome_16S.ali.fasta


MUSCLE v3.8.1551 by Robert C. Edgar

http://www.drive5.com/muscle
This software is donated to the public domain.
Please cite: Edgar, R.C. Nucleic Acids Res 32(5), 1792-97.

NCBI_cyanobiome_16S 31 seqs, lengths min 1292, max 2004, avg 1423
00:00:00      1 MB(0%)  Iter   1  100.00%  K-mer dist pass 1
00:00:00      1 MB(0%)  Iter   1  100.00%  K-mer dist pass 2
00:00:01     35 MB(0%)  Iter   1  100.00%  Align node       
00:00:01     35 MB(0%)  Iter   1  100.00%  Root alignment
00:00:01     37 MB(0%)  Iter   2  100.00%  Refine tree   
00:00:01     37 MB(0%)  Iter   2  100.00%  Root alignment
00:00:01     37 MB(0%)  Iter   2  100.00%  Root alignment
00:00:03     37 MB(0%)  Iter   3  100.00%  Refine biparts
00:00:05     37 MB(0%)  Iter   4  100.00%  Refine biparts
00:00:05     37 MB(0%)  Iter   5  100.00%  Refine biparts
00:00:05     37 MB(0%)  Iter   5  100.00%  Refine biparts


In [29]:
!/Users/tiagoferreiraleao/Dropbox/tiago-NAS/cyanobiome/barrnap/trimAl/source/trimal -in /Users/tiagoferreiraleao/Dropbox/tiago-NAS/cyanobiome/barrnap/NCBI_cyanobiome_16S.ali.fasta \
-out /Users/tiagoferreiraleao/Dropbox/tiago-NAS/cyanobiome/barrnap/NCBI_cyanobiome_16S.trimmed.fasta -gt 0.8 -st 0.001 -cons 70

Error: the symbol 'Y' accesing the matrix is not defined in this object
Error: the symbol 'Y' accesing the matrix is not defined in this object
Error: the symbol 'Y' accesing the matrix is not defined in this object
Error: the symbol 'Y' accesing the matrix is not defined in this object
Error: the symbol 'Y' accesing the matrix is not defined in this object
Error: the symbol 'Y' accesing the matrix is not defined in this object
Error: the symbol 'Y' accesing the matrix is not defined in this object
Error: the symbol 'Y' accesing the matrix is not defined in this object
Error: the symbol 'Y' accesing the matrix is not defined in this object
Error: the symbol 'Y' accesing the matrix is not defined in this object
Error: the symbol 'Y' accesing the matrix is not defined in this object
Error: the symbol 'Y' accesing the matrix is not defined in this object
Error: the symbol 'Y' accesing the matrix is not defined in this object
Error: the symbol 'Y' accesing the matrix is not defined in this

**Adding Name to Label**

In [30]:
record_dict = {}
input_list = glob.glob('/Users/tiagoferreiraleao/Dropbox/tiago-NAS/cyanobiome/barrnap/NCBI_symploca_16S_tree/*.txt')

for item in input_list:
    with open(item) as f:
        first_line = f.readline()
        ncbi_id = first_line.split(' ')[0][1:]
        taxa_info = ncbi_id + '_' + first_line.split(' ')[1] + '_' + first_line.split(' ')[2]
        record_dict[ncbi_id] = taxa_info
        
record_dict

{'AY172802.1': 'AY172802.1_Synechococcus_sp.',
 'NR_074317.1': 'NR_074317.1_Nostoc_punctiforme',
 'KF746599.1': 'KF746599.1_Caldora_penicillata',
 'GQ351563.1': 'GQ351563.1_Planktothrix_agardhii',
 'KF746602.1': 'KF746602.1_Caldora_penicillata',
 'KF746598.1': 'KF746598.1_Caldora_penicillata',
 'HF678501.1': 'HF678501.1_Anabaena_variabilis',
 'KF746607.1': 'KF746607.1_Caldora_penicillata',
 'LC534980.1': 'LC534980.1_Caldora_sp.',
 'EU315909.1': 'EU315909.1_Moorea_producens',
 'EF654039.1': 'EF654039.1_Coleofasciculus_chthonoplastes',
 'KF746603.1': 'KF746603.1_Caldora_penicillata',
 'KF746601.1': 'KF746601.1_Caldora_penicillata',
 'KF746590.1': 'KF746590.1_Caldora_penicillata',
 'KF746592.1': 'KF746592.1_Caldora_penicillata',
 'FJ041298.1': 'FJ041298.1_Moorea_bouillonii',
 'AY172836.1': 'AY172836.1_Synechococcus_sp.',
 'EF654054.1': 'EF654054.1_Coleofasciculus_chthonoplastes',
 'LC534997.1': 'LC534997.1_Caldora_sp.',
 'GQ402025.1': 'GQ402025.1_Symploca_sp.',
 'NR_074282.1': 'NR_074282.

In [32]:
new_file = []

for record in SeqIO.parse("./barrnap/NCBI_cyanobiome_16S.trimmed.fasta", "fasta"):
    if record.id in record_dict:
        record.id = record_dict[record.id]
    new_file.append(record)
output_handle = open("./barrnap/NCBI_cyanobiome_16S.ren.trimmed.fasta", "w")
SeqIO.write(new_file, output_handle, "fasta")
output_handle.close()

**Creating tree**

In [23]:
# !wget http://www.microbesonline.org/fasttree/FastTree -P /Users/tiagoferreiraleao/Dropbox/tiago-NAS/cyanobiome/barrnap/
!chmod -R 777 /Users/tiagoferreiraleao/Dropbox/tiago-NAS/cyanobiome/barrnap/FastTree

```
cd /Users/tiagoferreiraleao/Dropbox/tiago-NAS/cyanobiome/barrnap/

FastTree -nt < /Users/tiagoferreiraleao/Dropbox/tiago-NAS/cyanobiome/barrnap/NCBI_cyanobiome_16S.ren.trimmed.fasta > /Users/tiagoferreiraleao/Dropbox/tiago-NAS/cyanobiome/barrnap/concat_genes.tree
```